SUPPORT VECTOR MACHINES :

this is a type of supervised learning algo, 
they are powerful and versatile capable of handling complex decision boundaries making it suitable for a wide range of tasks 

linear classification : svm can separate data using a straight line in 2d or hyperplane 

non linear classificaion : using kernel tricks it can handle complex curved boundaries 

regression : svms can be adapted for regression tasks called SVR (support vector regressions)

novelty detection : svms can detect outliers and anomilies used in fraud detection or defect spotting 

they are good for small to medium datasets(hundreds to thousands) not millions, good for classification with a lot of features 

LINEAR SVM CLASSIFICATION :

SEE fig 5-1 :

left plot :
the dashed line doesnt even seperate the two classes properly 
pink and green lines seperate the classes correctly but they paas to some points very closely, they may not generalise well with the new data(overfitting issue)

right plot :
the solid black line is the decision boundary of the svm classifier and the dashed lines represent the margin lines 
svm maximizes the margin between the classes 
the suppors vectors(closest point in the decision boundary) lie on the dashboard margin lines 
this is called margin classification meaning the model tries to leave as much space between the two classess 
it doesnt just aim to seperate the classes but also finds the boundary that is furthest from any training point on either side 

if we add more training instances far from the margin the model wont change much as svm cares only about the closest points from each class 
the decision boundary is mathematically defined by few data points that lie on the margin, these are the most important instances for the model therefore called the support vectors these points are circled in fig 5-1 right plot 
all other points can be removed and the svm decision boundary wont change 

svms are highly sensitive to the scale of input features, if one feature has much larger range than other it can dominate the distance calc and that affects the decision boundary, seef fig 5-2

left plot :
x1 has larger range so the distance between the points is determined by differences in x1, resulting a near horizontal decision boundary, which results to a skewed street that doesnt reflect the true relationship betweeen the features 

right plot :
features are now standardized the decision boundaries are now diagonal which seperates the two classes better


see fig 5-3 :

hard margin classification :

svm strictly seperates the classes with a margin street this works only well when the data is perfectly linearly seperable, see fig 5-3 left plot :
an outlier is added this makes it impossible to draw a decision margin that seperates the two classes 

see fig 5-3 right plot :
in this example a hardl line is still attempted despite the outlier and the svm distorts the decision boundart to include that outlier 
the margin shrinks and generalisation will suffer, it wont perform well on unseen data

soft margin classification :
to avoid the above issue we need to use a more flexible model which will keep the street or margin as large as possible and limit the margin violations that means some will be misclassified but the majority will be inside the margin 


regularisation in svm :

we can use the svc class C param it controls the balance between the margin size and classification accuracy, see fig 5-4:

left plot: (c=1, soft margin)
larger margin(wider street), the svm allows more margin violations 
many support vectors are inside the margin(those with gray circles)
model is more prominent to errors less likely to overfit but it may fail to capture the real boundary

right plot (c=100, harder margin)
narrrower margin, model is trying to minimize the classification errors even if the margin shrinks
fewer margin violations are allowed (closer to a hard margin)
it may overfit but in this example it actually generalises better than the left model 

if your svm model is overfitting you can try regularize it by reducing the C

lets train a linear svm classifier now :

In [1]:
from sklearn.datasets import load_iris
iris  = load_iris(as_frame=True) #as_frame returns it as pandas dataframe

x = iris.data[["petal length (cm)", "petal width (cm)"]].values # a numpy array of two selected features

y = (iris.target == 2) #iris virginica
# a boolean label

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

svm_clf = make_pipeline(StandardScaler(), LinearSVC(C=1, random_state=42)) #c=1 soft margin

svm_clf.fit(x,y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, random_state=42))])

this pipeline fits the scalar and svm model on the training data 
the resulting decision boundary corresponds to left plot wider margin more violations 

lets make predictions now :

In [2]:
x_new = [[5.5, 1.7], [5.0, 1.5]]
svm_clf.predict(x_new)

array([ True, False])

In [3]:
svm_clf.decision_function(x_new)

array([ 0.66163816, -0.22035761])

decision_function() returns the signed distance between the decision boundary and point 
positive  = classified as true 
negative = classified as false (on the other side of the margin)


LinearSVC is optimized for speed and scalability,  it does not support predict_proba(estimated class probab) and predict_log_proba

instead we can use SCV(probability = True) to use both the methods 
working:
trains the svm model 
performs the 5 fold cross validations and get out-of-sample decision scores
fits a logistic reg on these scores, as a result output probab are caliberated using the logistic function 

it leads to slower training as it include some extra steps (CV and caliberation)